# Function calling testing

## 載入套件，建立 OpenAI 用戶端

In [2]:
from openai import OpenAI
client = OpenAI()

## 定義自訂函數(get_weather)的規格

In [ ]:
# 定義自訂函數(get_weather)的規格
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

## 測試

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

# 生成呼叫自訂函數的語法
print(response.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_GvFSfm7tQFNVunFvkdh3EOEc', function=Function(arguments='{"location":"Paris, France"}', name='get_weather'), type='function')]


## 定義自訂函數

In [13]:
import requests

# 定義自訂函數(get_weather)的規格，改以經緯度為參數
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

def get_weather(latitude, longitude):
    response = requests.get(
        "https://api.open-meteo.com/v1/forecast" + \
        f"?latitude={latitude}&longitude={longitude}&current=temperature_2m" + \
        ",wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data['current']['temperature_2m']

## 測試

In [ ]:
import json

messages = [{"role": "user", "content": "What's the weather like in Taipei today?"}]
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

response.choices[0].message.tool_calls

In [16]:
response.choices[0].message.tool_calls[0].function.arguments

'{"latitude":25.032969,"longitude":121.565418}'

In [17]:
tool_call = response.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = get_weather(args["latitude"], args["longitude"])
result

15.1